# Clustering

El objetivo de esta tarea es identificar productos que se comporten de manera similar para poderlos agrupar. De esta manera, se podrá evaluar de forma eficiente las campañas que se lleven a cabo y contribuir a un forecasting más exacto. 

# 1. Librerías

In [ ]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np
import pandas as pd

# loading ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# python core library for machine learning and data science
import sklearn
from sklearn import set_config
set_config(transform_output = "pandas")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

# data visualization
import seaborn as sns
import matplotlib.pyplot as plt


# 2. Carga el dataset

In [ ]:
RANDOM_STATE = 175

df_union = pd.read_csv("gb_union_weeks.csv")

### Genera el dataset para realizar la clusterización

In [ ]:
df = pd.DataFrame()

In [ ]:
df["item"] = df_union["item"].unique()

In [ ]:
df

# 3. Preprocesamiento de los datos

In [ ]:
df_union.info()

### Extraemos año y semana


In [ ]:
df_union["week"] = df_union["year_week"].astype(str).str[4:]

In [ ]:
df_union["year"] = df_union["year_week"].astype(str).str[:4]

In [ ]:
del(df_union["year_week"])

In [ ]:
# convertimos "week" en un número por separado
df_union["week"] = df_union["week"].astype(float)

In [ ]:
df_union["week"] = df_union["week"].astype(int)

In [ ]:
df_union["year"] = df_union["year"].astype(str)

### Pasamos "date" a fecha y mes a número

In [ ]:
df_union["date_d"] = pd.to_datetime(df_union["date"],format = '%Y-%m-%d')

In [ ]:
del(df_union["date"])

In [ ]:
df_union["month"] = df_union["date_d"].dt.strftime('%m')

In [ ]:
df_union["month"] = df_union["month"].astype(int)

In [ ]:
df_union.info()

### Tratamos los nulos

Sólo hay nulos "sell_price" debido a que las semanas que no hubo venta no hay un precio asignado. Se decide aplicar back y forward fill para completar con el precio que ese mismo producto tuvo anteriormente y, en su defecto, con el que tendrá en un futuro próximo. 


In [ ]:
df_union_ = df_union.copy() # realizamos una copia del dataset

In [ ]:
df_union_["sell_price_clean"] = df_union_.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill()
)

In [ ]:
del(df_union_["sell_price"])

In [ ]:
# comprobamos que los nulos hayan sido corregidos:
df_union_.isnull().sum()

# 4. Feature engineering

### Se calculan los ingresos semanales generados por producto 

In [ ]:
df_union_["revenue_semanal"]= df_union_["quantity"]* df_union_["sell_price_clean"]

### Se calcula el precio medio de cada item teniendo en cuenta todo el histórico de información

In [ ]:
df_union_["precio_medio"]= df_union_.groupby("item")["sell_price_clean"].transform("mean") 

###  Unidades vendidas por item

In [ ]:
# Unidades vendidas por producto en todas las tiendas y regiones, teniendo en cuenta todos los años 

df_union_["unidades_totales_item"]= df_union_.groupby("item")["quantity"].transform(sum)

# se creó esta función para calcular la performance, pero no resultó ser tan relevante para la clusterización
#def agregacion (df,list_variables,variable_agg,que):
#    cantidad_total = int(df[variable_agg].sum()) # 65600433
#    df[f"unidades_totales_por_{que}"]= df.groupby(list_variables)[variable_agg].transform(sum)
#    df[f"performance_unidades_{que}%"]= (df[f"unidades_totales_por_{que}"]/cantidad_total)*100
#    return df

In [ ]:
df_union_

### Ingresos por item

In [ ]:
# Ingresos generados por producto en todas las tiendas y regiones, teniendo en cuenta todos los años 

df_union_["ingreso_total_item"]= df_union_.groupby("item")["revenue_semanal"].transform(sum)


In [ ]:
df_union_

### Ingresos por estación

In [ ]:
#dividir estacionalmente los meses

def get_season(month):
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"
    else:
        return "Winter"

df_union_['estacion'] = df_union_['month'].apply(get_season)

In [ ]:
revenue_por_item_y_estacion = df_union_.groupby(['item', 'estacion'])['revenue_semanal'].sum()

In [ ]:
revenue_por_item_y_estacion = revenue_por_item_y_estacion.to_frame()

In [ ]:
pivot_revenue = revenue_por_item_y_estacion.pivot_table(index='item', columns='estacion', values= "revenue_semanal")

In [ ]:
# unimos la pivot table a la tabla creada en un inicio para el clustering
df_result = pd.merge(df, pivot_revenue, on="item", how="left") 

In [ ]:
df_result

### Ingresos por región

In [ ]:
revenue_por_item_y_region = df_union_.groupby(['item', 'region'])['revenue_semanal'].sum()

In [ ]:
revenue_por_item_y_region = revenue_por_item_y_region.to_frame()

In [ ]:
pivot_revenue = revenue_por_item_y_region.pivot_table(index='item', columns='region', values= "revenue_semanal")

In [ ]:
df_result = pd.merge(df_result, pivot_revenue, on="item", how="left") # unimos

In [ ]:
df_result

### Ingresos por categoría

In [ ]:
revenue_por_item_y_category = df_union_.groupby(['item', 'category'])['revenue_semanal'].sum()

In [ ]:
revenue_por_item_y_category = revenue_por_item_y_category.to_frame()

In [ ]:
pivot_revenue = revenue_por_item_y_category.pivot_table(index='item', columns='category', values= "revenue_semanal")

In [ ]:
df_result = pd.merge(df_result, pivot_revenue, on="item", how="left") # unimos

In [ ]:
df_result # da NaN y está bien porqué no todos los productos estan en todas las categorías

Trataremos los nuloc con 0 porqué no han generado ingresos para esa categoría puesto que pertenecen a otra.

In [ ]:
df_result["ACCESORIES"].fillna(0, inplace =True)

In [ ]:
df_result["HOME_&_GARDEN"].fillna(0, inplace =True)

In [ ]:
df_result["SUPERMARKET"].fillna(0, inplace =True)

### Ingresos por tienda

In [ ]:
revenue_por_item_y_store = df_union_.groupby(['item', 'store'])['revenue_semanal'].sum()

In [ ]:
revenue_por_item_y_store = revenue_por_item_y_store.to_frame()

In [ ]:
pivot_revenue = revenue_por_item_y_store.pivot_table(index='item', columns='store', values= "revenue_semanal")

In [ ]:
df_result = pd.merge(df_result, pivot_revenue, on="item", how="left")

In [ ]:
df_result

### Ingresos anuales 

In [ ]:
revenue_por_item_y_store = df_union_.groupby(['item', 'year'])['revenue_semanal'].sum()

In [ ]:
revenue_por_item_y_year = revenue_por_item_y_store.to_frame()

In [ ]:
revenue_por_item_y_year

In [ ]:
pivot_revenue = revenue_por_item_y_year.pivot_table(index='item', columns='year', values= "revenue_semanal")

In [ ]:
pivot_revenue

In [ ]:
df_result = pd.merge(df_result, pivot_revenue, on="item", how="left") # unimos

In [ ]:
df_result

# 5. Preparamos los datasets para el clustering

### Reducimos la granularidad  df_union_  a item

df_union_ es el dataset con la información por semanas preprocesado . 

In [ ]:
df_union_.columns.to_list()

In [ ]:
df_union_copy = df_union_.copy() # hacemos una copia del dataset

Eliminamos todas las que no están agrupadas a nivel de item

In [ ]:
COLUMNS_TO_DROP = ['id',
 'quantity',
 'category',
 'department',
 'store',
 'store_code',
 'region',
 'week',
 'year',
 'date_d',
 'month',
 'sell_price_clean',
 'revenue_semanal',
 'precio_medio',
 'estacion'
 ]

df_union_copy.drop(COLUMNS_TO_DROP, inplace = True, axis = 1)

In [ ]:
df_union_copy.drop_duplicates(inplace = True)

In [ ]:
df_union_copy.shape # comprobamos que sea el mismo número de rows que de items

In [ ]:
df_union_copy

### Creamos un dataset con la variable categoría

Creamos un nuevo dataset con el nombre de la categoría de cada producto. Este dataset lo utilizaremos después de hacer el clustering, pues el modelo no acepta variables categóricas

In [ ]:
df_union_after =  df_union_copy.copy()

In [ ]:
df_union_after["category_inletters"]= df_union_after["item"].astype(str).str[:11]

dict_tocategory = {
    'ACCESORIES_':"ACCESSORIES",
    'HOME_&_GARD':"HOME_&_GARDEN",
    'SUPERMARKET':"SUPERMARKET" 
}
df_union_after["category_inletters"] = df_union_after["category_inletters"].map(dict_tocategory)

In [ ]:
df_union_after

Sólo queremos conservar la columna de categoría, pues la información numérica ya la tenemos en el dataset del clúster. Así que borramos las columnas que no necesitamos para evitar duplicidad de información

In [ ]:
COLUMNS_TO_DROP = ['ingreso_total_item','unidades_totales_item'
 ]

df_union_after.drop(COLUMNS_TO_DROP, inplace = True, axis = 1)

In [ ]:
df_union_after

In [ ]:
df_union_after.set_index('item', drop=True, inplace = True)

### Hacemos el merge de los diferentes datasets para crear el dataset que utilizaremos para la clusterización

In [ ]:
df_result = pd.merge(df_result, df_union_copy, on="item", how="left") # unimos

In [ ]:
df_result.head(10)

In [ ]:
df_result.set_index('item', drop=True, inplace = True)

In [ ]:
df_result.shape

### Creamos la variable precio item 

In [ ]:
df_result["precio_item"]= df_result["ingreso_total_item"]/df_result["unidades_totales_item"]

In [ ]:
df_result

# 6. Clustering

### Elbow curve, para saber el número de clusters

Revisamos que ninguna variable sea categorica pues el modelo no las sabe gestionar. 

In [ ]:
df_result.info() 

In [ ]:
pipe = Pipeline(steps = [
    ("RobustScaler", RobustScaler(quantile_range = (0, 99.0)))

])

In [ ]:
df_scaled_transformed = pipe.fit_transform(df_result)

El SSE es una medida de la dispersión de los datos dentro de los clústeres. El resultado se almacena en el diccionario sse, con la clave "k" que indica el número de clústeres utilizados en el ajuste del modelo.
La inercia es nuestra métrica a medir: distancia de cada cliente a su cliente más cercano y luego haces una media

In [ ]:
sse = {} # se almacenan los valores de SSE de los diferentes clusters

for k in range(2, 20):
    
    print(f"Fitting pipe with {k} clusters")

    clustering_model = KMeans(n_clusters = k)
    clustering_model.fit(df_scaled_transformed)

    sse[k] = clustering_model.inertia_
    

In [ ]:
fig = plt.figure(figsize = (16, 8))
ax = fig.add_subplot()

x_values = list(sse.keys())
y_values = list(sse.values())

ax.plot(x_values, y_values, label = "Inertia/dispersión de los clústers")
fig.suptitle("Variación de la dispersión de los clústers en función de la k", fontsize = 16);

Como se puede apreciar los dos primeros codos que aparecen son entre el 3 y el 5. Así pues procedemos a elaborar nuestro clustering con 4 agrupaciones.

###   Segmentación de los productos con la "k" igual a 4

In [ ]:
pipe = Pipeline(steps = [ 
    ("RobustScaler", RobustScaler(quantile_range = (0, 99.0))),
    ("Clustering", KMeans(n_clusters = 4, random_state = 175))
])

In [ ]:
df_result.shape # comprobamos que tenemos tantas filas como número de items 3049

In [ ]:
pipe.fit(df_result)# nuestro modelo aprende a clasificar los productos en sus respectivos clústers

Creamos un nuevo dataset llamado X_processed, el qual es una copia de nuestro dataset df_result. Hacemos esta acción para poder guardar una versión de nuestros datos no escalados. 

In [ ]:
X_processed = df_result.copy() 

In [ ]:
labels = pipe.predict(df_result) # el modelo aprende, pero no le aplicamos el transform para poder trabajar con los valores sin escalar

In [ ]:
labels # la predicción del cluster

In [ ]:
X_processed["cluster"] = labels # añadimos la columna de predicción a nuestro dataset

In [ ]:
X_processed.head(3)

Unimos la variable categórica creada anteriormente, en el dataset df_union_after, al dataset ya clusterizado, X_processed

In [ ]:
X_processed = pd.merge(X_processed, df_union_after, on= "item", how="left")

In [ ]:
X_processed

# 7. Interpretación de los Clusters 

In [ ]:
pd.set_option('display.max_rows', None)
X_processed.groupby(["cluster"]).describe().T.style.background_gradient(cmap = 'Blues', axis = 1)

Después de interpretar los clústers se les asigna un nombre descriptivo a cada uno en función de sus características

In [ ]:
dict_clusters = {
    0:"Esporádicos",
    1:"Emocional",
    2:"Premium",
    3:"Esencial"
}

X_processed['cluster_name'] = X_processed['cluster'].map(dict_clusters)

# 8. Creación de excel para PowerBI

In [ ]:
#CSV con toda la información
X_processed.to_excel('X_processed_4.2.xlsx', index=True)

## Creación de dataframes adhoc para powerBI 

### Ingresos por año y cluster

In [ ]:
X_processed_copy = X_processed.copy()

Creamos un dataset con el nombre de item y clúster para incorporarlo en el dataset que contiene la información por semanas

In [ ]:
X_processed_copy.columns.to_list()

In [ ]:
COLUMNS_TO_DROP = ['Autumn',
 'Spring',
 'Summer',
 'Winter',
 'Boston',
 'New York',
 'Philadelphia',
 'ACCESORIES',
 'HOME_&_GARDEN',
 'SUPERMARKET',
 'Back_Bay',
 'Brooklyn',
 'Greenwich_Village',
 'Harlem',
 'Midtown_Village',
 'Queen_Village',
 'Roxbury',
 'South_End',
 'Tribeca',
 'Yorktown',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 'unidades_totales_item',
 'ingreso_total_item',
 'precio_item',
 'category_inletters'
 ]

X_processed_copy.drop(COLUMNS_TO_DROP, inplace = True, axis = 1)

In [ ]:
X_processed_copy

In [ ]:
df_union_.head(2)

In [ ]:
df_union_clusters = pd.merge(df_union_,X_processed_copy,  on= "item", how="left")

df_union_ es el dataset con la información por semanas, preprocesado y con feature engineering. A este dataset le hacemos un merge con X_processed_copy, que tiene las etiquetas de cluster para cada item. 

In [ ]:
df_union_clusters["cluster"].unique() # comprobamos que todos los clusters esten representados

In [ ]:
# generamos un groupby con la información que queremos visualizar
ingresos_año_cluster = df_union_clusters.groupby(['cluster_name','item', 'year'])['revenue_semanal'].sum()
ingresos_año_cluster

In [ ]:
#transformamos el groupby en un dataframe
ingresos_año_cluster = ingresos_año_cluster.to_frame()

In [ ]:
ingresos_año_cluster.reset_index(inplace=True)

In [ ]:
ingresos_año_cluster.rename(columns={'revenue_semanal': 'revenue_anual'}, inplace=True)

In [ ]:
ingresos_año_cluster.to_excel('df_ingresos_año_cluster.xlsx', index=True)

### Dataset categoría

In [ ]:
df_category = df_union_clusters.groupby(['cluster_name','item','category'])['revenue_semanal'].sum()

In [ ]:
df_category= df_category.to_frame()

In [ ]:
df_category.reset_index(inplace=True) # movemos los indices a columnas

In [ ]:
df_category.rename(columns={'revenue_semanal': 'revenue_category'}, inplace=True)

In [ ]:
df_category.head(3)

In [ ]:
df_category.to_excel('df_category.xlsx', index=True)

### Dataset región y tienda

In [ ]:
df_region_store = df_region_store = df_union_clusters.groupby(['cluster_name','region','store'])['revenue_semanal'].sum()

In [ ]:
df_region_store = df_region_store.to_frame()

In [ ]:
df_region_store.reset_index(inplace=True) # movemos los indices a columnas

In [ ]:
df_region_store.rename(columns={'revenue_semanal': 'revenue_location'}, inplace=True)

In [ ]:
df_region_store.head(5)

In [ ]:
df_region_store.to_excel('df_region_store.xlsx', index=True)

### Dataset estación

In [ ]:
df_estación = df_estación = df_union_clusters.groupby(['cluster_name','estacion',"year"])['revenue_semanal'].sum()
df_estación = df_estación.to_frame()

In [ ]:

df_estación.reset_index(inplace=True)
df_estación.rename(columns={'revenue_semanal': 'revenue_estación'}, inplace=True)

In [ ]:
df_estación.to_excel('df_estación.xlsx', index=True)